# Ensemble Methods

## 1. Ensemble의 의미  
  
  : 여러 개의 모델을 결합하여 하나의 모델보다 더 좋은 성능을 내는 머신러닝 기법이다.

## 2. Errors
  
  : Model에서 나오는 Error는 아래와 같이 3가지 부분으로 나뉘어진다.

![LmeI08b.png](https://i.imgur.com/LmeI08b.png)

- Error가 중요한 이유  
: Ensemble 모델의 바탕을 이해하기 위해서 모델에 있어서 어떤 것이 error를 발생시키는 지를 알아야 한다.

#### Bias Error  
  
  : **predicted value가 actual value와 평균적으로 얼마나 차이가 있는지**를 나타내는 값이다. Bias Error가 높다는 것은 어떤 중요한 경향성에 대한 파악을 하지 못하고 있어 model의 성능이 낮다(underfitting)는 것을 의미한다.

#### Variance  
  
  : **같은 관측치에 대한 prediction이 다른 관측치에 대한 것과 얼마나 다른지**를 나타내는 값이다. 모델의 Variance가 높다는 것은 train에 대해 overfit 되어있어 train data가 아닌 다른 값에 대해서는 성능을 보이지 않는 것을 의미한다. 

![jFfarvo.png](https://i.imgur.com/jFfarvo.png)

따라서 모델은 이 두가지 error에 대한 balance를 유지하여야 한다. 이것이 잘 알려진 bias-variance trade off에 대한 것이며 **Ensemble**은 이러한 trade off 문제를 해결할 수 있는 방법이다.

## 3. Ensemble의 종류

### Basic Ensemble Method  
  
  > Max Voting  
  > Averaging  
  > Weighted Average

#### 1) Max Voting  
  
  : max voting은 분류 문제에 주로 쓰인다.  
  여러 개의 모델이 각각 각자의 data에 대한 예측을 진행하고 각각의 예측은 하나의 "투표(vote)"가 된다.  
  예측들 간에 투표를 진행하여 다수결에 의해 최종 예측을 하게 된다.

``` python
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

model1 = LogisticRegression(random_state=1)
model2 = DecisionTreeClassifier(random_state=1)
model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')

"""
### Parameters ###

- estimator
 : (모델명, 모델)의 튜플들을 리스트로 감싼 형태로 인자를 받는다.
- voting
 : 'hard'는 단순 투표이며, 'soft'는 가중치 투표, 즉 각 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 선택한다.
"""

model.fit(x_train,y_train)
model.score(x_test,y_test)
```

#### - Hard Voting  
  
  : 가장 단순한 형태의 다수결 투표  
  - y = mode{C1(x),C2(x),...,Cm(x)}  
  - predict()

#### - Soft Voting  
  
  : 각 클래스의 확률들의 평균을 내어 가장 높은 확률의 클래스를 선택  
  높은 확률의 모델에 대해 가중치를 더 주므로 hard voting보다 더 높은 성능을 보인다.
  - y = argmaxi∑j = 1mwjpij
  - predict_proba()  
SVC의 경우 default는 class의 확률을 반환하지 않기 때문에 probability hyperparameter를 True로 설정해 주고 predict_proba() method를 사용하여야 한다.

#### 2) Averaging  
  
  : max voting과 비슷하게 각 data에 대해 생성된 여러 개의 예측를 모두 평균을 취하는 방법이다. 이 방법은 Regression 문제나 Classification 문제에서 확률을 측정해야 할 때에 사용된다.

``` python
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

model1 = DecisionTreeClassifier()
model2 = KNeighborsClassifier()
model3 = LogisticRegression()

model1.fit(x_train,y_train)
model2.fit(x_train,y_train)
model3.fit(x_train,y_train)

pred1 = model1.predict_proba(x_test)
pred2 = model2.predict_proba(x_test)
pred3 = model3.predict_proba(x_test)

finalpred = (pred1+pred2+pred3)/3
# 세 예측의 확률을 평균을 취해 최종 예측의 확률로 선택한다.
```

#### 3) Weighted Average  
  
  : 모든 모델에 대해 다른 가중치를 주어 더 중요한 모델의 예측에 더 높은 가중치를 주는 방법이다. 

``` python
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

model1 = DecisionTreeClassifier()
model2 = KNeighborsClassifier()
model3 = LogisticRegression()

model1.fit(x_train,y_train)
model2.fit(x_train,y_train)
model3.fit(x_train,y_train)

pred1 = model1.predict_proba(x_test)
pred2 = model2.predict_proba(x_test)
pred3 = model3.predict_proba(x_test)

finalpred = (pred1*0.3+pred2*0.3+pred3*0.4)
# 세 예측의 확률에 가중치를 주어 최종 예측의 확률로 선택한다.
```

### Advanced Ensemble Method  
  
  > Stacking  
  > Blending  
  > Bagging  
  > Boosting

### 1) Bagging 
##### (Bootstrap Aggregation)
  
  : data에 variance가 존재해야 한다. 즉, 샘플을 여러 번 뽑아(Bootstrap) 각 모델을 학습시켜 결과물을 집계 (Aggregration)하는 방법이다. 동일한 모델을 사용하여 데이터만 분할하여 여러개 모델을 학습한다.

##### - Stability와 Accuracy  
  
  : 각 prediction들의 평균을 취할 때 bias에 영향을 주지 않고 variance를 낮출 뿐만 아니라 accuracy도 높여주게 된다. 약간씩 다른 모델들을 ensemble하게 되면 overfitting을 피하면서 예측을 안정화시키고 accuracy를 높일 수 있다.  
  :  **이 때 data에 variance가 필수!**

##### - 방법  

1. 복원추출을 통해 데이터에 대한 여러 subset들을 생성
2. 각 subset들을 모델에 적용
3. 각 모델들은 병렬적, 독립적으로 학습
4. 각 모델로 test data 예측
5. 각 모델들의 예측 집계 (voting이나 averaging 방법) 후 최종 예측 생성

![eu95V9N.png](https://i.imgur.com/eu95V9N.png)

#### Bagging의 Algorithms
  
  > Bagging meta estimator  
  > Random Forest  

#### Bagging meta-estimator  
  
  : classification 문제 (BaggingClassifier)와 regression 문제 (BaggingRegressor) 둘다에 쓰일 수 있는 방법이다. 예측을 할때 Bagging 방법을 따른다.

##### - 방법  

1. 데이터에서 랜덤하게 subset들을 생성 (Bootstrapping)
2. 각 subset들은 모든 feature를 포함
3. base_estimator를 설정하여 작은 set들에 fit
4. 각 모델로 부터 나온 예측을 합쳐 최종 예측 생성

``` python
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

final_dt = DecisionTreeClassifier(max_leaf_nodes=10, max_depth=5)         
final_bc = BaggingClassifier(base_estimator=final_dt, n_estimators=40, random_state=1, oob_score=True)

final_bc.fit(X_train, train_y)
final_preds = final_bc.predict(X_test)

acc_oob = final_bc.oob_score_
print(acc_oob)  

"""
### Parameters ###

- base_estimator
 : 랜덤 subset에 fit시킬 모델 설정 (default는 decision tree)
- n_estimators
 : base_estimator의 개수
- max_samples
 : 각 subset sample의 최대 개수를 의미
- max_features
 : 데이터의 feature 최대 개수를 의미
"""
```

- OOB(Out-of-Bag) 평가  
배깅은 중복을 허용하는 리샘플링(resampling) 즉, 부트스트래핑(bootstraping) 전체 학습 데이터셋에서 어떠한 데이터 샘플은 여러번 샘플링 되고, 또 어떠한 샘플은 전혀 샘 플링 되지 않을 수가 있다. 평균적으로 학습 단계에서 전체 학습 데이터셋 중 63% 정도만 샘플 링 되며, 샘플링 되지 않은 나머지 37% 데이터 샘플들을 oob(out-of-bag) 샘플이라고 한다.  
앙상블(배깅) 모델의 학습 단계에서는 oob 샘플이 사용되지 않기 때문에, 이러한 oob 샘플을 검증셋(validation set)이나 교차검증(cross validation)에 사용할 수 있다.

#### Random Forest  
  
  : 마찬가지로 bagging 방법을 따른다. base estimator는 decision tree이며 bagginb meta estimator와 다르게 random forest는 각 모델이 어떤 feature가 가장 최적의 분기를 하는지를 선택할 때 feature의 set을 random하게 선택한다.  
  : 즉, 랜덤하게 행과 열을 선택하고 그 데이터로 여러 tree들을 생성하여 forest를 이룬다.

##### - 방법  

1. 데이터에서 랜덤하게 subset들을 생성 (Bootstrapping)
2. decision tree가 어떤 feature가 가장 최적의 분기를 하는지를 선택할 때 feature의 set을 random하게 선택
3. 한 decision tree가 각각의 subset에 fit
4. 각 모델로부터 나온 예측을 평균을 취해 최종 예측 생성

> 행과 열 모두에서 랜덤하게 선택할 수 있다.
> sklearn의 경우 decision tree에서 모든 feature를 사용하며 각 노드에서 분기할 때 feature의 subset은 랜덤하게 선택된다.

``` python
# Classification Problem
from sklearn.ensemble import RandomForestClassifier

mode l= RandomForestClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test, y_test)

# Regression Problem
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(x_train, y_train)
model.score(x_test, y_test)

"""
### Parameters ###

- n_estimators
 : random forest에서 몇 개의 decision tree를 생성할지
 보통 개수가 많을 수록 더 좋은 예측 성능을 보이지만 그 시간 또한 매우 오래걸린다.
- criterion
 : 분기를 할때 사용되는 함수
 'entropy' 또는 'gini'가 사용된다.
- max_features
 : 각 decision tree가 분기를 할 때 사용되는 최대 feature 개수
 클수록 각 tree의 성능이 좋아지지만 tree들의 다양성은 줄어든다.
- max_depth
 : tree들의 최대 깊이
- min_samples_split
 : leaf node에 도달하기 전 요구되는 sample의 최소 개수
- min_samples_leaf
 : leaf node에 요구되는 sample의 최소 개수
- max_leaf_nodes
 : 각 tree의 최대 leaf node 개수
"""
```

### 2) Boosting 
  
  : 가중치를 활용하여 약 분류기를 강 분류기로 만드는 방법이다. 처음 모델이 예측을 하면 그 예측 결과에 따라 데이터에 가중치가 부여되고, 부여된 가중치가 다음 모델에 영향을 준다. 잘못 분류된 데이터에 집중(가중치 부여)하여 새로운 분류 규칙을 만드는 단계를 반복한다.

#### Boosting의 Algorithms  
  > AdaBoost  
  > GBM  
  > XGB  
  > Light GBM  
  > CatBoost

#### AdaBoost
##### (Adaptive boosting)
  
  : 주로 decision tree가 모델로 사용된다. 모델이 그 전 모델로부터 나온 error를 잘 분류해내는 방법으로 순차적으로 모델들이 생성된다. 잘못 분류된 데이터에 가중치를 주어 그 다음 모델은 해당 값에 집중하여 더 잘 예측할 수 있도록 한다.

##### - 방법  

1. 각 데이터들은 처음에 모두 동일한 가중치를 가짐
2. 데이터의 subset으로부터 하나의 모델 생성
3. 이 모델을 이용해 모든 dataset에 대해 예측
4. 예측과 실제 값을 비교하여 error 계산
5. 잘못 예측된 데이터에 대해 더 높은 가중치를 준 상태로 다음 모델 생성
6. 가중치는 error 값에 의해 결정됨 (높은 error 값의 데이터에 더 높은 가중치를 줌)
7. error function이 바뀌지 않을 때까지 또는 최대 estimator의 수에 도달할 때까지 반복

``` python
# Classification Problem
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test, y_test)

# Regression Problem
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor()
model.fit(x_train, y_train)
model.score(x_test, y_test)

"""
### Parameters ###

- base_estimators
 : base 모델을 무엇을 사용할지 (default는 max_depth=1인 decision tree)
- n_estimators
 : base estimator의 개수 (default는 10개이며 클수록 좋은 성능을 보인다.)
- learning_rate
 : 각 estimator의 기여도를 결정하는 parameter
 learning_rate와 n_estimators는 trade-off 관계를 가진다.
- max_depth
 : 각 estimator의 최대 깊이
"""
```

### 3) Stacking
  
  : 각 모델들에 대해 원본 데이터를 학습시킨다. Boosting과 비슷하지만 차이는 meta-level 방식의 접근으로 input에 대한 예측을 진행하고 나온 모델들의 output들을 새로운 모델에 학습시킨다는 것이다.

- data를 다음과 같이 training, validation, test로 나눈다.

![EQYa8C8.png](https://i.imgur.com/EQYa8C8.png)

- train data(A)에 algorithm 0을 학습시키고 validataion set, test set (B, C)에 대한 예측을 진행하여 B1, C1에 저장한다.

![fVazCYe.png](https://i.imgur.com/fVazCYe.png)
![2zjxVBC.png](https://i.imgur.com/2zjxVBC.png)

- train data(A)에 algorithm 1을 학습시키고 validataion set, test set (B, C)에 대한 예측을 진행하여 B1, C1에 저장한다. 이때 예측들은 stack된다.

![lpDhGd1.png](https://i.imgur.com/lpDhGd1.png)
![10slay8.png](https://i.imgur.com/10slay8.png)

- train data(A)에 algorithm 2을 학습시키고 validataion set, test set (B, C)에 대한 예측을 진행하여 B1, C1에 저장한다.

![hfp6JGP.png](https://i.imgur.com/hfp6JGP.png)

- algorithm 3로 모든 예측들을 stack시킨 B1를 학습하고 C1에 대한 예측을 진행한다.

![md3L8yB.png](https://i.imgur.com/md3L8yB.png)

``` python
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split

# split train data in 2 parts
x_train, x_valid, y_train, y_valid = train_test_splid(train, y, test_size=0.5)

# specify models
model1 = RandomForstRegressor()
model2 = LinearRegression()

# fit models
model1.fit(x_train, y_train)
model2.fit(x_train, y_train)

# make prediction for validation
preds1 = model1.predict(x_valid)
preds2 = model2.predict(x_valid)

# make prediction for test
test_preds1 = model1.predict(test)
test_preds2 = model2.predict(test)

# form a new dataset for valid and test via stacking the predictions
stacked_predictions = np.column_stack((preds1, preds2))
stacked_test_predictions = np.column_stack((test_preds1, test_preds2))

# specify meta model
meta_model = LinearRegression()

# fit meta model
meta_model.fit(stacked_predictions, y_valid)

# make predictions for stacked predictions of the test data
final_predictions = meta_model.predict(stacked_test_predictions)
```